In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END, START
from typing import TypedDict, Literal
from pydantic import BaseModel, Field

In [3]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash")

In [5]:
class SentimentSchema(BaseModel):
    sentiment: Literal["positive","negative"] = Field(description="sentiment of the review")

In [ ]:
class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')

In [14]:
struc_llm = llm.with_structured_output(SentimentSchema)
struc_llm2 = llm.with_structured_output(DiagnosisSchema)

In [16]:
struc_llm2.invoke("hii")

DiagnosisSchema(issue_type='Other', tone='calm', urgency='low')

In [9]:
class ReviewState(TypedDict):
    review: str
    sentiment: Literal["positive","negative"]
    response: str
    diagnosis: str

In [ ]:
def find_sentiment(state:ReviewState):
    prompt = f"Find the sentiment of the following review: \n {state["review"]}"
    output = struc_llm.invoke(prompt).sentiment

    return {"sentiment":output}

def cheackpoint(state: ReviewState)-> Literal["positive_response","run_diagnosis"]:
    if state["review"] == "positive":
        return "positive_response"
    else:
        return "run_diagnosis"
    
def positive_response(state: ReviewState):

    prompt = f'''Write a warm thank-you to the response of the review: 
    {state["review"]}.
    and kindly ask for feedback on our website'''
    
    output = llm.invoke(prompt).content
    return {"response":output}

def run_diagnosis(state:ReviewState):

    prompt = f"""Diagnose this negative review:\n\n{state['review']}\n"
    "Return issue_type, tone, and urgency.
"""
    response = struc_llm2.invoke(prompt)

    return {'diagnosis': response.model_dump()}

def negative_response(state: ReviewState):

    diagnosis = state['diagnosis']

    prompt = f"""You are a support assistant.
The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'.
Write an empathetic, helpful resolution message.
"""
    response = llm.invoke(prompt).content

    return {'response': response}

In [20]:
graph = StateGraph(ReviewState)

graph.add_node("find_sentiment",find_sentiment)
graph.add_node("positive_response",positive_response)
graph.add_node("run_diagnosis",run_diagnosis)
graph.add_node("negative_response",negative_response)

graph.add_edge(START,"find_sentiment")
graph.add_conditional_edges("find_sentiment",cheackpoint)

graph.add_edge("positive_response",END)
graph.add_edge("run_diagnosis","negative_response")
graph.add_edge("negative_response",END)

workflow = graph.compile()


In [ ]:
w